In [3]:
from numpy.polynomial.legendre import legval, legvander
import numpy as np

# モデル関数: f(x) = x + 2
def model(x):
    return x + 2

# サンプル点（[-1, 1] の一様分布を想定）
N = 100
x = np.random.uniform(-1, 1, N)
y = model(x)

# Legendre多項式の基底（次数2まで）
deg = 2
V = legvander(x, deg)  # Vandermonde matrix: 各点での多項式評価

# 最小二乗で係数を推定
coeffs, *_ = np.linalg.lstsq(V, y, rcond=None)

print("PCE（Legendre基底）係数:", coeffs)


PCE（Legendre基底）係数: [2.00000000e+00 1.00000000e+00 5.79404092e-18]


In [15]:
import numpy as np

B_dash = [[1, 2], [3, 4]]
# 2x2 のブロック行列を用意
B = np.array(B_dash)
C = np.array([[5, 6], [7, 8]])
D = np.array([[9, 10], [11, 12]])
E = np.array([[13, 14], [15, 16]])

# ブロックを2x2に配置してnp.blockでまとめる
A = np.block([[B, C],
              [D, E]])

print(A)

[[ 1  2  5  6]
 [ 3  4  7  8]
 [ 9 10 13 14]
 [11 12 15 16]]


In [13]:
import numpy as np

# 2x2 のブロック行列を用意
B = np.array([[1, 2], [3, 4]])
C = np.array([[5, 6], [7, 8]])
D = np.array([[9, 10], [11, 12]])
E = np.array([[13, 14], [15, 16]])

# ブロックを2x2に配置してnp.blockでまとめる
A = np.block([[B, C],
              [D, E]])

print(A)


[[ 1  2  5  6]
 [ 3  4  7  8]
 [ 9 10 13 14]
 [11 12 15 16]]


In [1]:
import numpy as np
import cvxpy as cp
from scipy.linalg import block_diag

# 例として小さな系を用意
n = 2   # 状態数
m = 1   # 入力数
p = 1   # カオス次数 → (p+1)で拡張次元に拡張される

# システム行列（任意の値に設定可能）
A = np.array([[0.5, 1.0],
              [-1.0, -2.0]])
B = np.array([[1.0],
              [0.0]])

Ip = np.eye(p + 1)
A_big = np.kron(np.eye(p + 1), A)
B_big = np.kron(np.eye(p + 1), B)

# 重み行列（任意）
Q = np.eye(n)
R = np.eye(m)
W = np.eye(p + 1)
Qx_bar = np.kron(Q, W)
Ru_bar = np.kron(R, W)

# Kをランダムに初期化（実際は反復で更新）
K = np.random.randn(m, n)

# K_full = K ⊗ Ip
K_big = np.kron(K, Ip)

# 変数 P（対称行列）
dim = n * (p + 1)
P = cp.Variable((dim, dim), symmetric=True)

# 式(45)の左辺
LHS = A_big.T @ P + P @ A_big + \
      P @ B_big @ K_big + K_big.T @ B_big.T @ P + \
      Qx_bar + K_big.T @ Ru_bar @ K_big

# 制約と目的関数の定義
constraints = [P >> 1e-3 * np.eye(dim)]
objective = cp.Minimize(cp.norm(LHS, 'fro'))

# 最適化
prob = cp.Problem(objective, constraints)
prob.solve()

print("最適化成功:", prob.status)
print("P =", P.value)

最適化成功: optimal_inaccurate
P = [[ 397.21677768  478.49101376  797.32173022  891.99920852]
 [ 478.49101376  577.55145074  960.60931871 1076.61193091]
 [ 797.32173022  960.60931871 1600.73745859 1791.32366476]
 [ 891.99920852 1076.61193091 1791.32366476 2008.10697898]]


c:\Users\shun-\AppData\Local\Programs\Python\Python312\Lib\site-packages\cvxpy\problems\problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
